# Fine Metrics Demo: Complete Guide to Dask Spans Integration

This notebook demonstrates all fine-grained performance metrics available through Dask Spans in roastcoffea.

## What are Fine Metrics?

Fine metrics provide detailed performance breakdowns beyond wall time:
- **CPU vs Non-CPU time**: How much time spent computing vs waiting/coordination?
- **Processor vs Overhead**: Separate your processor work from Dask coordination overhead
- **Compression overhead**: Time spent compressing/decompressing data
- **Serialization overhead**: Time spent serializing/deserializing Python objects
- **Disk/Memory I/O**: Bytes read from disk or memory
- **Real compression ratios**: Actual uncompressed vs compressed bytes

## Important: Processor Instance Parameter

To get accurate metrics separating your processor work from Dask overhead, **always pass `processor_instance`** to `MetricsCollector`:

```python
processor = MyProcessor()
with MetricsCollector(client, processor_instance=processor) as collector:
    output, report = runner(fileset, processor_instance=processor)
```

Without it, all metrics (including Dask coordination overhead) are aggregated together.

## Data Sources

Fine metrics come from **Dask Spans** and are available at multiple granularities:
1. **Processor-level**: Your actual processor work (when processor_instance provided)
2. **Overhead-level**: Dask coordination tasks (lambdas, etc.)
3. **Per-task**: Broken down by individual tasks (task prefix)
4. **Per-worker**: (Future) Separated by individual workers

## Setup: Install and Import

In [5]:
# Standard imports

import awkward as ak

# Get a test file from scikit-hep-testdata
import skhep_testdata
from coffea import processor
from coffea.nanoevents import NanoAODSchema
from dask.distributed import Client, LocalCluster

# roastcoffea imports
from roastcoffea import (
    MetricsCollector,
    plot_per_task_bytes_read,
    # Per-task visualization
    plot_per_task_cpu_io,
    plot_per_task_overhead,
)

test_file = skhep_testdata.data_path("nanoAOD_2015_CMS_Open_Data_ttbar.root")
print(f"Using test file: {test_file}")

Using test file: /Users/moaly/.local/skhepdata/nanoAOD_2015_CMS_Open_Data_ttbar.root


## Define a Simple Coffea Processor

We'll create a processor that does some real work: filtering jets, computing masses, and creating histograms.

In [6]:
class JetAnalysisProcessor(processor.ProcessorABC):
    """Simple jet analysis processor for testing fine metrics."""

    def process(self, events):
        # Select jets with pT > 30 GeV
        jets = events.Jet[events.Jet.pt > 30]

        # Select events with at least 2 jets
        two_jet_events = events[ak.num(jets) >= 2]
        two_jets = jets[ak.num(jets) >= 2]

        # Calculate dijet invariant mass for leading two jets
        if len(two_jets) > 0:
            j1 = two_jets[:, 0]
            j2 = two_jets[:, 1]
            dijet_mass = (j1 + j2).mass
        else:
            dijet_mass = ak.Array([])

        return {
            "nevents": len(events),
            "njets_total": ak.sum(ak.num(jets)),
            "nevents_2jet": len(two_jet_events),
            "dijet_mass_mean": ak.mean(dijet_mass) if len(dijet_mass) > 0 else 0,
            "jet_pt_sum": ak.sum(jets.pt),
        }

    def postprocess(self, accumulator):
        return accumulator

## Part 1: Collecting Fine Metrics

Let's run the processor with metrics collection enabled. Fine metrics are collected automatically when Dask Spans are available.

In [7]:
# Create fileset
fileset = {
    "DY": {
        "files": {test_file: "Events"},
    },
}

# Start Dask cluster
cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True)
client = Client(cluster)

print(f"Dashboard: {client.dashboard_link}")

/Users/moaly/Work/iris-hep/roastcoffea/.pixi/envs/dev/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 65146 instead
  warnings.warn(


Dashboard: http://127.0.0.1:65146/status


In [ ]:
# Run with metrics collection - RECOMMENDED approach
instance = JetAnalysisProcessor()
with MetricsCollector(
    client, track_workers=True, processor_instance=instance
) as collector:
    executor = instance.DaskExecutor(client=client)
    runner = instance.Runner(
        executor=executor,
        savemetrics=True,
        schema=NanoAODSchema,
    )

    output, report = runner(
        fileset,
        treename="Events",
        processor_instance=instance,
    )

    # Provide report to collector
    collector.set_coffea_report(report)

print("\n" + "=" * 60)
print("Analysis complete!")
print("=" * 60)

Dask Span completed but no metrics were collected. Fine-grained metrics may not be available.


AttributeError: 'JetAnalysisProcessor' object has no attribute 'DaskExecutor'

## Part 2: Viewing Cumulative Fine Metrics

Let's see the workflow-level metrics aggregated across all tasks and workers.

In [ ]:
# Print comprehensive summary (includes fine metrics table)
collector.print_summary()

                        Throughput Metrics                        
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metric            ┃ Value                                      ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Data Rate         │ 0.00 Gbps (0.4 MB/s)                       │
│ Compression Ratio │ 0.00x                                      │
│ Total Data Read   │ 328.73 KB compressed, 1.51 KB uncompressed │
└───────────────────┴────────────────────────────────────────────┘

           Event Processing Metrics           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric                     ┃ Value         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Total Events               │ 200           │
│ Event Rate (Wall Clock)    │ 0.2 kHz       │
│ Event Rate (Aggregated)    │ 5.0 kHz       │
│ Event Rate (Core-Averaged) │ 120.0 Hz/core │
│ Efficiency Ratio           │ 4.8%          │
└────────────────────────────┴───────────────┘

          Resource Utilization          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Metric                   ┃ Value     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ Workers (Time-Averaged)  │ 2.0       │
│ Peak Workers             │ 2         │
│ Cores per Worker         │ 1.0       │
│ Total Cores              │ 2         │
│ Core Efficiency          │ 2.4%      │
│ Speedup Factor           │ 0.0x      │
│ Peak Memory (per worker) │ 216.58 MB │
│ Avg Memory (per worker)  │ 117.72 MB │
└──────────────────────────┴───────────┘

       Timing Breakdown       
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric             ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Wall Time          │ 0.8s  │
│ Total CPU Time     │ 0.0s  │
│ Number of Chunks   │ 1     │
│ Avg CPU Time/Chunk │ 0.0s  │
└────────────────────┴───────┘

 Fine Metrics (from 
    Dask Spans)     
┏━━━━━━━━━━┳━━━━━━━┓
┃ Metric   ┃ Value ┃
┡━━━━━━━━━━╇━━━━━━━┩
│ CPU Time │ 0.2s  │
│ I/O Time │ 0.0s  │
│ CPU %    │ 93.3% │
│ I/O %    │ 6.7%  │
└──────────┴───────┘

In [ ]:
# Access fine metrics programmatically
metrics = collector.get_metrics()

print("\n=== Key Fine Metrics ===")
if metrics.get("processor_cpu_time_seconds") is not None:
    print(f"Processor CPU Time: {metrics['processor_cpu_time_seconds']:.2f} seconds")
    print(
        f"Processor Non-CPU Time: {metrics['processor_noncpu_time_seconds']:.2f} seconds"
    )
    print(f"Processor CPU %: {metrics['processor_cpu_percentage']:.1f}%")
    print(f"Processor Non-CPU %: {metrics['processor_noncpu_percentage']:.1f}%")
    print()

    # Show overhead metrics if available
    overhead_cpu = metrics.get("overhead_cpu_time_seconds", 0)
    overhead_noncpu = metrics.get("overhead_noncpu_time_seconds", 0)
    if overhead_cpu > 0 or overhead_noncpu > 0:
        print("=== Dask Overhead ===")
        print(f"Overhead CPU Time: {overhead_cpu:.2f} seconds")
        print(f"Overhead Non-CPU Time: {overhead_noncpu:.2f} seconds")
        print()

    print(f"Disk Read: {metrics.get('disk_read_bytes', 0) / 1e9:.2f} GB")
    print(f"Disk Write: {metrics.get('disk_write_bytes', 0) / 1e9:.2f} GB")
    print()

    compression_overhead = metrics.get("total_compression_overhead_seconds", 0)
    if compression_overhead > 0:
        print(f"Compression Overhead: {compression_overhead:.2f} seconds")
        print(f"  - Decompress: {metrics.get('decompression_time_seconds', 0):.2f}s")
        print(f"  - Compress: {metrics.get('compression_time_seconds', 0):.2f}s")
        print()

    serialization_overhead = metrics.get("total_serialization_overhead_seconds", 0)
    if serialization_overhead > 0:
        print(f"Serialization Overhead: {serialization_overhead:.2f} seconds")
        print(f"  - Deserialize: {metrics.get('deserialization_time_seconds', 0):.2f}s")
        print(f"  - Serialize: {metrics.get('serialization_time_seconds', 0):.2f}s")
        print()

    if (
        metrics.get("compression_ratio") is not None
        and metrics["compression_ratio"] > 0
    ):
        print(f"Compression Ratio: {metrics['compression_ratio']:.2f}x")
        print(
            f"Uncompressed Bytes: {metrics.get('total_bytes_uncompressed', 0) / 1e9:.2f} GB"
        )
else:
    print("Fine metrics not available (Dask Spans may not be enabled)")

## Part 3: Understanding the Data Structure

Let's look at the raw Span metrics to understand the data format.

In [ ]:
# Access raw span metrics
span_metrics = collector.span_metrics

if span_metrics:
    print("=== Raw Span Metrics Structure ===")
    print(f"Total keys: {len(span_metrics)}")
    print("\nSample keys (first 10):")
    for i, (key, value) in enumerate(list(span_metrics.items())[:10]):
        print(f"  {key}: {value}")

    print("\n=== Key Format Explanation ===")
    print("Keys are tuples: (context, task_prefix, activity, unit)")
    print("  - context: 'execute' for task execution")
    print("  - task_prefix: Task identifier (e.g., 'process-abc123')")
    print("  - activity: 'thread-cpu', 'thread-noncpu', 'disk-read', etc.")
    print("  - unit: 'seconds' for time, 'bytes' for data size")
else:
    print("No span metrics collected")

=== Raw Span Metrics Structure ===
Total keys: 11

Sample keys (first 10):
  ('execute', 'lambda', 'thread-cpu', 'seconds'): 1.7542000000000002e-05
  ('execute', 'lambda', 'thread-noncpu', 'seconds'): 1.4644508178710935e-05
  ('execute', 'lambda', 'executor', 'seconds'): 0.00034168828278779984
  ('execute', 'lambda', 'other', 'seconds'): 0.000291542150080204
  ('execute', 'JetAnalysisProcessor', 'memory-read', 'count'): 1
  ('execute', 'JetAnalysisProcessor', 'memory-read', 'bytes'): 1546
  ('execute', 'JetAnalysisProcessor', 'thread-cpu', 'seconds'): 0.234355125
  ('execute', 'JetAnalysisProcessor', 'thread-noncpu', 'seconds'): 0.01683911366271973
  ('execute', 'JetAnalysisProcessor', 'executor', 'seconds'): 0.0009408453479409218
  ('execute', 'JetAnalysisProcessor', 'other', 'seconds'): 0.00030058296397328377

=== Key Format Explanation ===
Keys are tuples: (context, task_prefix, activity, unit)
  - context: 'execute' for task execution
  - task_prefix: Task identifier (e.g., 'proces

## Part 4: Per-Task Analysis

Fine metrics are broken down by task prefix, allowing us to see which tasks are CPU-bound vs waiting/coordination-bound.

**Note:** "Non-CPU time" includes waiting time, GIL contention, coordination overhead, and other non-compute activities - not just I/O operations.

In [ ]:
# Extract per-task breakdown
from roastcoffea.visualization.plots.per_task import extract_per_task_metrics

if span_metrics:
    per_task = extract_per_task_metrics(span_metrics)

    print("=== Per-Task Metrics ===")
    print(f"Number of unique tasks: {len(per_task)}")
    print("\nTask breakdown:")

    for task_prefix, activities in per_task.items():
        print(f"\n  Task: {task_prefix}")
        cpu = activities.get("thread-cpu", 0)
        noncpu = activities.get("thread-noncpu", 0)
        total = cpu + noncpu

        if total > 0:
            print(f"    CPU time: {cpu:.3f}s ({cpu / total * 100:.1f}%)")
            print(f"    Non-CPU time: {noncpu:.3f}s ({noncpu / total * 100:.1f}%)")

        if "disk-read" in activities:
            print(f"    Disk read: {activities['disk-read'] / 1e6:.2f} MB")
        if "memory-read" in activities:
            print(f"    Memory read: {activities['memory-read'] / 1e6:.2f} MB")

        if "decompress" in activities:
            print(f"    Decompress time: {activities['decompress']:.3f}s")
        if "deserialize" in activities:
            print(f"    Deserialize time: {activities['deserialize']:.3f}s")

## Part 5: Per-Task Visualizations

Let's create plots showing the distribution across tasks.

In [ ]:
# Plot CPU vs Non-CPU time per task
if span_metrics:
    try:
        fig, ax = plot_per_task_cpu_io(
            span_metrics=span_metrics,
            title="CPU vs Non-CPU Time per Task",
            figsize=(14, 6),
        )
        print("Successfully created CPU/Non-CPU plot")
    except ValueError as e:
        print(f"Could not create CPU/Non-CPU plot: {e}")

In [ ]:
# Plot bytes read per task (disk-read or memory-read)
if span_metrics:
    try:
        fig, ax = plot_per_task_bytes_read(
            span_metrics=span_metrics, title="Bytes Read per Task", figsize=(14, 6)
        )
        print("Successfully created bytes read plot")
    except ValueError as e:
        print(f"Could not create bytes read plot: {e}")

Could not create bytes read plot: No per-task disk-read metrics found in span_metrics


In [ ]:
# Plot compression & serialization overhead per task
if span_metrics:
    try:
        fig, ax = plot_per_task_overhead(
            span_metrics=span_metrics,
            title="Compression & Serialization Overhead per Task",
            figsize=(14, 6),
        )
        print("Successfully created overhead plot")
    except ValueError as e:
        print(f"Could not create overhead plot: {e}")

Could not create overhead plot: No per-task overhead metrics found in span_metrics


## Part 6: Comparing Scenarios

Let's run multiple scenarios to see how fine metrics help identify bottlenecks.

**Note on Robustness:** roastcoffea automatically handles metric synchronization timing. Dask Spans sync metrics from workers to the scheduler via heartbeats (default: 1s interval). Our implementation uses retry logic with exponential backoff to ensure metrics are available even if tasks complete right after a heartbeat.

In [ ]:
# Helper function to run and collect metrics
def run_analysis(processor_class, label):
    """Run analysis and return fine metrics."""
    processor = processor_class()
    with MetricsCollector(
        client, track_workers=False, processor_instance=processor
    ) as collector:
        executor = processor.DaskExecutor(client=client)
        runner = processor.Runner(
            executor=executor,
            savemetrics=True,
            schema=NanoAODSchema,
        )

        output, report = runner(
            fileset,
            treename="Events",
            processor_instance=processor,
        )

        collector.set_coffea_report(report)
    metrics = collector.get_metrics()

    return {
        "label": label,
        "cpu_time": metrics.get("processor_cpu_time_seconds", 0),
        "noncpu_time": metrics.get("processor_noncpu_time_seconds", 0),
        "cpu_pct": metrics.get("processor_cpu_percentage", 0),
        "noncpu_pct": metrics.get("processor_noncpu_percentage", 0),
        "wall_time": metrics.get("wall_time", 0),
        "overhead_cpu": metrics.get("overhead_cpu_time_seconds", 0),
        "overhead_noncpu": metrics.get("overhead_noncpu_time_seconds", 0),
    }


# Define a CPU-heavy processor
class CPUHeavyProcessor(processor.ProcessorABC):
    """Processor with more CPU-intensive operations."""

    def process(self, events):
        jets = events.Jet
        # Compute multiple derived quantities
        for _ in range(5):  # Repeat calculations
            pt = jets.pt
            eta = jets.eta
            phi = jets.phi
            energy = ak.zip({"pt": pt, "eta": eta, "phi": phi})
        return {"nevents": len(events)}

    def postprocess(self, accumulator):
        return accumulator


# Run both scenarios
print("Running analysis scenarios...\n")
results = []
results.append(run_analysis(JetAnalysisProcessor, "Standard Analysis"))
results.append(run_analysis(CPUHeavyProcessor, "CPU-Heavy Analysis"))

# Compare results
print("\n=== Scenario Comparison ===")
print(
    f"{'Scenario':<25} {'Wall Time':<12} {'Proc CPU':<12} {'Proc Non-CPU':<14} {'CPU %':<8} {'Non-CPU %':<10}"
)
print("-" * 95)
for r in results:
    print(
        f"{r['label']:<25} {r['wall_time']:<12.2f} {r['cpu_time']:<12.2f} {r['noncpu_time']:<14.2f} {r['cpu_pct']:<8.1f} {r['noncpu_pct']:<10.1f}"
    )

print("\n=== Dask Overhead (if separated) ===")
print(f"{'Scenario':<25} {'Overhead CPU':<15} {'Overhead Non-CPU':<18}")
print("-" * 65)
for r in results:
    if r["overhead_cpu"] > 0 or r["overhead_noncpu"] > 0:
        print(
            f"{r['label']:<25} {r['overhead_cpu']:<15.3f} {r['overhead_noncpu']:<18.3f}"
        )
    else:
        print(f"{r['label']:<25} {'N/A':<15} {'N/A':<18}")

## Part 7: Understanding Worker-Level Data (Future)

Currently, we collect **cumulative** metrics aggregated across all workers. However, Dask also tracks per-worker metrics in `Worker.digests_total`.

### Current Implementation
- **`Span.cumulative_worker_metrics`**: Aggregated across all workers
- Keys: `('execute', task_prefix, activity, unit)`

### Available (Not Yet Implemented)
- **`Worker.digests_total`**: Per-worker metrics with span IDs
- Keys: `('execute', span_id, task_prefix, activity, unit)`
- Would allow per-worker CPU/I/O analysis

### What This Means
If we access worker digests directly, we could answer questions like:
- Which workers are CPU-bound vs I/O-bound?
- Is work balanced across workers?
- Do some workers have more compression overhead?

This is a potential future enhancement!

## Part 8: Key Takeaways

### What Fine Metrics Tell You

1. **CPU % high**: Your analysis is compute-bound
   - Optimize algorithms
   - Consider vectorization with NumPy/Awkward operations
   - May benefit from more cores

2. **Non-CPU % high**: Your analysis is waiting/coordination-bound
   - Includes I/O, waiting time, GIL contention, task coordination
   - Optimize data access patterns
   - Consider columnar filtering
   - May not benefit from more cores

3. **High Dask overhead**: Coordination tasks consuming significant time
   - Consider larger chunk sizes
   - Reduce task graph complexity
   - May indicate too many small tasks

4. **High compression overhead**: Decompression is a bottleneck
   - Consider different compression codecs
   - Pre-decompress if reading same files repeatedly

5. **High serialization overhead**: Python object pickling is slow
   - Minimize object complexity
   - Use simpler data structures
   - Consider Dask serialization optimizations

6. **Low compression ratio**: Data not compressing well
   - Already compressed formats
   - Random/entropy-heavy data

### Granularity Levels

- **Cumulative**: Overall workflow performance
- **Processor-separated**: Actual processor work vs Dask overhead (when processor_instance provided)
- **Per-task**: Identify outlier tasks or unbalanced work
- **Per-worker** (future): Diagnose worker-specific issues

### Limitations

- No function-level profiling (use `cProfile` or `py-spy` for that)
- Activities are predefined by Dask (can't add custom metrics)
- Requires recent Dask version with Spans support
- "Non-CPU" is a catch-all metric - includes many different waiting/coordination scenarios

## Cleanup

In [ ]:
# Close cluster
client.close()
cluster.close()
print("Cluster closed")